# Kaggle提出用ファイルを作成する

# 1.S3からforecast結果をdownload

In [2]:
!aws s3 cp --recursive s3://work-kaggle-ohio/store-sales-time-series-forecasting/export_targetonly_noholiday/ ../result_forecast_targetonly_noholiday/

download: s3://work-kaggle-ohio/store-sales-time-series-forecasting/export_targetonly_noholiday/targetonly_noholiday_2021-12-22T08-43-36Z_part0.csv to ../result_forecast_targetonly_noholiday/targetonly_noholiday_2021-12-22T08-43-36Z_part0.csv
download: s3://work-kaggle-ohio/store-sales-time-series-forecasting/export_targetonly_noholiday/_CHECK to ../result_forecast_targetonly_noholiday/_CHECK
download: s3://work-kaggle-ohio/store-sales-time-series-forecasting/export_targetonly_noholiday/targetonly_noholiday_2021-12-22T08-43-36Z_part1.csv to ../result_forecast_targetonly_noholiday/targetonly_noholiday_2021-12-22T08-43-36Z_part1.csv
download: s3://work-kaggle-ohio/store-sales-time-series-forecasting/export_targetonly_noholiday/targetonly_noholiday_2021-12-22T08-43-36Z_part102.csv to ../result_forecast_targetonly_noholiday/targetonly_noholiday_2021-12-22T08-43-36Z_part102.csv
download: s3://work-kaggle-ohio/store-sales-time-series-forecasting/export_targetonly_noholiday/_SUCCESS to ../res

## データ結合

In [2]:
import glob
import pandas as pd

df = pd.DataFrame()

files = glob.glob("../result_forecast_targetonly_noholiday/*.csv")
for file in files:
    #print(file)
    df_tmp = pd.read_csv(file)
    df = pd.concat([df,df_tmp]).reset_index(drop=True)

In [3]:
df

,item_id,date,store_nbr,p10,p50,p90
0,beauty,2017-08-16T00:00:00Z,25,0.750166,2.789812,4.635010
1,beauty,2017-08-17T00:00:00Z,25,-0.135176,2.517841,4.884029
2,beauty,2017-08-18T00:00:00Z,25,0.326958,4.131533,7.308528
3,beauty,2017-08-19T00:00:00Z,25,0.329145,3.246138,7.528454
4,beauty,2017-08-20T00:00:00Z,25,0.874725,4.269231,6.614688
...,...,...,...,...,...,...
26725,seafood,2017-08-26T00:00:00Z,5,4.574191,10.245558,14.601978
26726,seafood,2017-08-27T00:00:00Z,5,5.298670,11.445668,16.342670
26727,seafood,2017-08-28T00:00:00Z,5,1.914506,8.073844,12.953550
26728,seafood,2017-08-29T00:00:00Z,5,1.306258,7.617987,12.674770


## 提出用ファイルに整形

In [4]:
df_sub = pd.read_csv('../input/sample_submission.csv')

In [5]:
df_sub

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


In [20]:
df_test = pd.read_csv('../input/test.csv')

In [25]:
print(df_test.nunique())
print(df_test['date'].min())
print(df_test['date'].max())
print(54*33*16)

id             28512
date              16
store_nbr         54
family            33
onpromotion      212
dtype: int64
2017-08-16
2017-08-31
28512


# !!!!!! 予測期間16日じゃないかあああああああ

In [13]:
df_test['item_id'] = df_test['family'].str.lower()

In [14]:
df_test

,id,date,store_nbr,family,onpromotion,item_id
0,3000888,2017-08-16,1,AUTOMOTIVE,0,automotive
1,3000889,2017-08-16,1,BABY CARE,0,baby care
2,3000890,2017-08-16,1,BEAUTY,2,beauty
3,3000891,2017-08-16,1,BEVERAGES,20,beverages
4,3000892,2017-08-16,1,BOOKS,0,books
...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,poultry
28508,3029396,2017-08-31,9,PREPARED FOODS,0,prepared foods
28509,3029397,2017-08-31,9,PRODUCE,1,produce
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,school and office supplies


In [15]:
df.head()

,item_id,date,store_nbr,p10,p50,p90
0,beauty,2017-08-16T00:00:00Z,25,0.750166,2.789812,4.635010
1,beauty,2017-08-17T00:00:00Z,25,-0.135176,2.517841,4.884029
2,beauty,2017-08-18T00:00:00Z,25,0.326958,4.131533,7.308528
3,beauty,2017-08-19T00:00:00Z,25,0.329145,3.246138,7.528454
4,beauty,2017-08-20T00:00:00Z,25,0.874725,4.269231,6.614688


In [16]:
df['date'] = df['date'].str.replace('T',' ').str.replace(' 00:00:00Z','')

In [17]:
df['sales'] = df['p50']

In [18]:
df.head()

,item_id,date,store_nbr,p10,p50,p90,sales
0,beauty,2017-08-16,25,0.750166,2.789812,4.635010,2.789812
1,beauty,2017-08-17,25,-0.135176,2.517841,4.884029,2.517841
2,beauty,2017-08-18,25,0.326958,4.131533,7.308528,4.131533
3,beauty,2017-08-19,25,0.329145,3.246138,7.528454,3.246138
4,beauty,2017-08-20,25,0.874725,4.269231,6.614688,4.269231


In [19]:
pd.merge(df, df_test[['date','item_id','store_nbr','id']], how='left', on=['date','item_id','store_nbr'])

,item_id,date,store_nbr,p10,p50,p90,sales,id
0,beauty,2017-08-16,25,0.750166,2.789812,4.635010,2.789812,3001451
1,beauty,2017-08-17,25,-0.135176,2.517841,4.884029,2.517841,3003233
2,beauty,2017-08-18,25,0.326958,4.131533,7.308528,4.131533,3005015
3,beauty,2017-08-19,25,0.329145,3.246138,7.528454,3.246138,3006797
4,beauty,2017-08-20,25,0.874725,4.269231,6.614688,4.269231,3008579
...,...,...,...,...,...,...,...,...
26725,seafood,2017-08-26,5,4.574191,10.245558,14.601978,10.245558,3020192
26726,seafood,2017-08-27,5,5.298670,11.445668,16.342670,11.445668,3021974
26727,seafood,2017-08-28,5,1.914506,8.073844,12.953550,8.073844,3023756
26728,seafood,2017-08-29,5,1.306258,7.617987,12.674770,7.617987,3025538
